In [ ]:
import pandas as pd
import docker
client = docker.from_env()
from nlputils import utils
from nlputils.components.axaserver import axaprocessor

In [ ]:
# read the files dataframe created from load_docs.ipynb
df_pdf = pd.read_json("folder_location")\
                    .query('ImagePDF.notna() & ImagePDF==False').reset_index(drop=True)
df_pdf['pdf_path'] = df_pdf.apply(lambda x: x['orig_filepath'] if x['filetype'] == 'pdf' 
                                  else x['converted_filepath'],axis=1)

# keeping image pdf as separate dataframe
df_imagepdf = pd.read_json("folder_location")\
                    .query('ImagePDF.notna() & ImagePDF==True').reset_index(drop=True)
df_imagepdf['pdf_path'] = df_imagepdf.apply(lambda x: x['orig_filepath'] if x['filetype'] == 'pdf' 
                                  else x['converted_filepath'],axis=1)

# Document Processing

Sort the files with page count:
- For file_size < 10 pages: keep batch_size= 8-10, batch_wait_time= 300 sec
- 10 < For file_size < 25 pages: keep batch_size= 5-10, batch_wait_time= 300 sec
- 25 < For file_size < 60 pages: keep batch_size= 5-10, batch_wait_time= 600 sec
- 60 < For file_size: keep batch_size < 5, batch_wait_time = 1200-1800 sec

In [ ]:
# instantiate class object and pass the files list
batch_test= axaprocessor.axaBatchProcessingLocal(container_id="bf132427ed0273fc6b0b0744486df975eff9fd7856f18bd46c91ba987547723b",
                                                 batch_files=df_pdf.pdf_path.tolist())
# set the batch parameters, depending on memeory and number of cores that docker has access to
batch_test.set_batch_params()
batch_test.processing("save_to_folder_location")

Process ImagePDF's

In [ ]:
# instantiate class object and pass the files list
batch_test= axaprocessor.axaBatchProcessingLocal(container_id="bf132427ed0273fc6b0b0744486df975eff9fd7856f18bd46c91ba987547723b",
                                                 batch_files=df_imagepdf.pdf_path.tolist(), config='ocr')
# set the batch parameters, depending on memeory and number of cores that docker has access to
batch_test.set_batch_params(batch_size=7,batch_wait_time=600)
batch_test.processing("save_to_folder_location")